In [17]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import time
import api_keys
import requests
import gdax
import random as random
import json
import pandas as pd

api = api_keys.api_keys()

client = MongoClient(api.mongo)
# specify the database and collection`
db = client.gdax.gdaxws
print("connected to mongo collection")

# and store in local variables
key = api.api_key
b64secret = api.secret_key
passphrase = api.passphrase

class clearingmaster():
    def __init__(self, maxopenbuyorders = 4 ,**kwargs):
        self.maxopenbuyorders = maxopenbuyorders
        self.heartbeat_rate = 10  # seconds before we auto cancel limit order
        self.marketBTCUSD = client.get_product_ticker("BTC-USD")
        self.marketBTCEUR = client.get_product_ticker("BTC-EUR")
        self.exchangerate = float(self.marketBTCEUR["price"]) / float(self.marketBTCUSD["price"])

    def getorders(self):
        """{'id': 'e22c9172-0276-47f7-b774-2559784c26aa', 'price': '999.85000000', 
            'size': '0.01000000', 'product_id': 'BTC-EUR', 'side': 'buy', 
            'type': 'limit', 'time_in_force': 'GTC', 'post_only': True,
            'created_at': '2018-08-17T14:57:18.551778Z', 'fill_fees': '0.0000000000000000',
            'filled_size': '0.00000000', 'executed_value': '0.0000000000000000', 
            'status': 'open', 'settled': False}"""
    
        openorders = client.get_orders()[0]
        try:
            df_openorders = pd.DataFrame(openorders).query('product_id == "BTC-EUR"')
            df_openorders['created_at'] = pd.to_datetime(df_openorders['created_at'])
            self.df_openorders = df_openorders
            return True
        except:
            print('DAXY no orders on books')
            self.df_openorders = pd.DataFrame(None)
            return False
        
    def getbalances(self):
        """{'id': '459d001f-0391-4e97-89e7-ae474275e2c9', 'currency': 'BTC',
            'balance': '0.0530287336346057', 'available': '0.0530287336346057',
            'hold': '0.0000000000000000', 'profile_id': '5100622b-3ed2-49e4-9810-c28fb96d30b3'} """

        balances = client.get_accounts()
        df_balances = pd.DataFrame(balances).query('currency in ("BTC","EUR")').T
        df_balances.rename(columns=df_balances.loc['currency'], inplace=True)
        
        l = ['available','balance','hold']        
        self.df_balances = df_balances.loc[l].astype('float')
        #print('DAXY DEBUG', self.df_balances)

    def getclearance(self, kwargs_dict):
        try:
            orderprice = kwargs_dict['price'] * kwargs_dict['size']
            print('DAXY CM order received total price: {}'.format(orderprice))
            self.getbalances()
            self.getorders()
        except KeyError:
            print('DAXY CM KeyError calculating orderprice')
            print('DAXY CM KeyError rejecting order request')
            return False

        if self.df_balances['EUR']['available'] <= orderprice:
            print('DAXY CM insufficient funds')
            return False
        elif len(self.df_openorders) >= self.maxopenbuyorders:
            print('DAXY CM maximum limit orders reached')
            return False
        else:
            return True

    def heartbeat(self, **kwargs):
        has_orders = self.getorders()
        if has_orders is True:
            cutoffdate = pd.to_datetime(time.time()-self.heartbeat_rate, unit='s')
            to_terminate = self.df_openorders[self.df_openorders["created_at"]<cutoffdate]['id']
            for idtoterminate in to_terminate:
                response_cancel = client.cancel_order(idtoterminate) 
                db.delete_one({'MONGOKEY':'BUY_ORDER',
                               'trade_id': idtoterminate})
                print('DAXY CANCELLED old order : {}'.format(response_cancel))
        else:
            print('DAXY no orders on books')

class GAC(gdax.AuthenticatedClient):
    def __clearingmaster__(self):
        self.cm_ = clearingmaster()
        self.timeout = 15

    def buy(self, **kwargs):
        """client.buy(size="0.005000000",
                product_id="BTC-EUR",
                side="buy",
                stp="dc",
                type="limit")
                
            from CB PRO website
                {"id": "d0c5340b-6d6c-49d9-b567-48c4bfca13d2",
                "price": "0.10000000",
                "size": "0.01000000",
                "product_id": "BTC-USD",
                "side": "buy",
                "stp": "dc",
                "type": "limit",
                "time_in_force": "GTC",
                "post_only": false,
                "created_at": "2016-12-08T20:02:28.53864Z",
                "fill_fees": "0.0000000000000000",
                "filled_size": "0.00000000",
                "executed_value": "0.0000000000000000",
                "status": "pending",
                "settled": false}"""

        kwargs["side"] = "buy"
        kwargs["type"] = "limit"
        trade_request = self.cm_.getclearance(kwargs_dict=kwargs)
        
        if trade_request == True:
            r = requests.post(self.url + '/orders',
                            data=json.dumps(kwargs),
                            auth=self.auth,
                            timeout=30)

            rjson = r.json()
            rjson.update({'MONGOKEY':'BUY_ORDER','timestamp':time.time(),'trade_id':rjson['id']})
            rjson.pop('id', None)
            db.insert_one(rjson)
            print('DAXY BOUGHT inserted in mongo')
            return rjson
        else:
            print('DAXY CM no permission')

    def sell(self, **kwargs):
        """client.sell(size="0.005000000",
                product_id="BTC-EUR",
                side="sell",
                stp="dc",
                type="limit")"""

        kwargs["side"] = "sell"
        r = requests.post(self.url + '/orders',
                          data=json.dumps(kwargs),
                          auth=self.auth,
                          timeout=30)

        print('SOLD !!')
        print(r)
        r.update({'MONGOKEY':'SELL_ORDER','timestamp':time.time()})
        db.insert_one(r.json())
        print('DAXY SOLD inserted in mongo')
        return r.json()

if __name__ == "__main__":
    client = GAC(key=key,b64secret=b64secret,
                 passphrase=passphrase,
                 api_url="https://api.pro.coinbase.com")
    
    client.__clearingmaster__() # set clearing master
    client.cm_.getorders()
    p_change = {}
    counter=0

connected to mongo collection


In [6]:
client = GAC(key=key,b64secret=b64secret,
                 passphrase=passphrase,
                 api_url="https://api.pro.coinbase.com")
    
client.__clearingmaster__() # set clearing master
client.cm_.getorders()

DAXY no orders on books


False

In [8]:
client.buy(size=0.005000000,price=5671,
                product_id="BTC-EUR",
                side="buy",
                type="limit")

DAXY CM order received total price: 28.355
DAXY no orders on books
DAXY BOUGHT inserted in mongo


{'MONGOKEY': 'BUY_ORDER',
 '_id': ObjectId('5b77771e2d105a04fc3ed063'),
 'created_at': '2018-08-18T01:32:13.817883Z',
 'executed_value': '0.0000000000000000',
 'fill_fees': '0.0000000000000000',
 'filled_size': '0.00000000',
 'post_only': False,
 'price': '5671.00000000',
 'product_id': 'BTC-EUR',
 'settled': False,
 'side': 'buy',
 'size': '0.00500000',
 'status': 'pending',
 'stp': 'dc',
 'time_in_force': 'GTC',
 'timestamp': 1534555934.291138,
 'trade_id': 'f4d0cb7c-6ce8-4093-9bc5-4fe969f4d210',
 'type': 'limit'}

In [20]:
client.cm_.getorders()
client.cm_.df_openorders

,created_at,executed_value,fill_fees,filled_size,id,post_only,price,product_id,settled,side,size,status,time_in_force,type
0,2018-08-18 13:52:24.090197,0.0000000000000000,0.0000000000000000,0.00000000,6ddd73ad-7fc2-44b0-ad06-d471d515a17c,False,5714.59000000,BTC-EUR,False,sell,0.00500000,open,GTC,limit
1,2018-08-18 13:51:58.558703,0.0000000000000000,0.0000000000000000,0.00000000,a5ec009d-dd8a-4638-9b97-4f37fe543984,False,5719.78000000,BTC-EUR,False,sell,0.00900000,open,GTC,limit
2,2018-08-18 13:51:57.736098,0.0000000000000000,0.0000000000000000,0.00000000,92789b9f-4858-4418-b872-314b73eb33fd,False,5714.59000000,BTC-EUR,False,sell,0.01100000,open,GTC,limit
3,2018-08-18 13:51:57.564594,0.0000000000000000,0.0000000000000000,0.00000000,494decac-a63f-45cc-ae47-edb14d6fa9b3,False,5715.57000000,BTC-EUR,False,sell,0.00500000,open,GTC,limit
4,2018-08-18 13:51:24.116280,0.0000000000000000,0.0000000000000000,0.00000000,c400c60c-0648-4d7a-9fe2-652a22b636d8,False,5712.54000000,BTC-EUR,False,sell,0.00200000,open,GTC,limit
5,2018-08-18 13:51:23.222479,0.0000000000000000,0.0000000000000000,0.00000000,7026a075-640a-4163-866f-c4eb41a40a3c,False,5707.36000000,BTC-EUR,False,sell,0.01100000,open,GTC,limit
6,2018-08-18 13:51:23.157064,0.0000000000000000,0.0000000000000000,0.00000000,421092b7-9a95-41a6-8bd8-3b009eb84e99,False,5708.33000000,BTC-EUR,False,sell,0.00800000,open,GTC,limit
7,2018-08-18 13:50:52.358367,0.0000000000000000,0.0000000000000000,0.00000000,c8d606af-16e2-4524-bdad-62495f87663c,False,5712.54000000,BTC-EUR,False,sell,0.00300000,open,GTC,limit
8,2018-08-18 13:50:51.909229,0.0000000000000000,0.0000000000000000,0.00000000,cdbeeed9-a66c-4377-9e0c-f336af161563,False,5707.36000000,BTC-EUR,False,sell,0.01100000,open,GTC,limit
9,2018-08-18 13:50:51.759155,0.0000000000000000,0.0000000000000000,0.00000000,2b117b8c-819b-4d10-8fe8-1c3f1d7aee60,False,5708.33000000,BTC-EUR,False,sell,0.00300000,open,GTC,limit


In [21]:
(client.cm_.df_openorders['price'].astype('float') * client.cm_.df_openorders['size'].astype('float')).sum()

388.40556

In [23]:
df = pd.DataFrame(
        list(db.find({'MONGOKEY' :'BUY_ORDER'})\
                        .sort([('timestamp', 1)]))) 
df

,MONGOKEY,_id,created_at,executed_value,fill_fees,filled_size,post_only,price,product_id,settled,side,size,status,stp,strategy,time_in_force,timestamp,trade_id,type
0,BUY_ORDER,5b7772f262cf8b0001655492,2018-08-18T01:14:26.867631Z,0.0000000000000000,0.0000000000000000,0.00000000,False,5766.94000000,BTC-EUR,False,buy,0.00200000,pending,dc,ISS,GTC,1.534555e+09,e81a1d03-9250-43e7-8df3-39bb56825028,limit
1,BUY_ORDER,5b7772f308993e000142a236,2018-08-18T01:14:26.938084Z,0.0000000000000000,0.0000000000000000,0.00000000,False,5773.83000000,BTC-EUR,False,buy,0.00200000,pending,dc,ISS,GTC,1.534555e+09,b527f967-d0aa-48a6-b699-54ef831b85e3,limit
2,BUY_ORDER,5b7772f362cf8b0001655493,2018-08-18T01:14:27.224835Z,0.0000000000000000,0.0000000000000000,0.00000000,False,5766.94000000,BTC-EUR,False,buy,0.00200000,pending,dc,ISS,GTC,1.534555e+09,df2f689a-41f4-48e4-9114-76d056c0ab86,limit
3,BUY_ORDER,5b7772f308993e000142a237,2018-08-18T01:14:27.629932Z,0.0000000000000000,0.0000000000000000,0.00000000,False,5773.83000000,BTC-EUR,False,buy,0.00100000,pending,dc,ISS,GTC,1.534555e+09,61b5cd23-b59a-480e-9671-43bf9e593a18,limit
4,BUY_ORDER,5b7772f4b33e410001a614c1,2018-08-18T01:14:28.115748Z,0.0000000000000000,0.0000000000000000,0.00000000,False,5766.94000000,BTC-EUR,False,buy,0.00400000,pending,dc,ISS,GTC,1.534555e+09,81955b3c-a687-4eb9-a1cd-f1b219cfe20a,limit
5,BUY_ORDER,5b7772f4b33e410001a614c2,2018-08-18T01:14:28.557229Z,0.0000000000000000,0.0000000000000000,0.00000000,False,5766.94000000,BTC-EUR,False,buy,0.00400000,pending,dc,ISS,GTC,1.534555e+09,5f2ff4c3-47c3-41b2-a835-ff274adee7fa,limit
6,BUY_ORDER,5b7772f462cf8b0001655494,2018-08-18T01:14:28.748189Z,0.0000000000000000,0.0000000000000000,0.00000000,False,5766.94000000,BTC-EUR,False,buy,0.00200000,pending,dc,ISS,GTC,1.534555e+09,f3abdf0d-61e3-4be3-9b8d-c173f1fb8d02,limit
7,BUY_ORDER,5b7772f5b33e410001a614c3,2018-08-18T01:14:28.930809Z,0.0000000000000000,0.0000000000000000,0.00000000,False,5766.94000000,BTC-EUR,False,buy,0.00400000,pending,dc,ISS,GTC,1.534555e+09,ec0f804c-4b47-40d2-9f8a-646a1ec7735f,limit
8,BUY_ORDER,5b7772f5b33e410001a614c4,2018-08-18T01:14:29.388901Z,0.0000000000000000,0.0000000000000000,0.00000000,False,5766.94000000,BTC-EUR,False,buy,0.00100000,pending,dc,ISS,GTC,1.534555e+09,28d4b3be-2d19-47d8-bbb9-2e0e228748c1,limit
9,BUY_ORDER,5b7772f508993e000142a238,2018-08-18T01:14:29.414598Z,0.0000000000000000,0.0000000000000000,0.00000000,False,5773.83000000,BTC-EUR,False,buy,0.00400000,pending,dc,ISS,GTC,1.534555e+09,f6e72842-540b-4ac2-9a77-682fa9375890,limit


In [28]:
df[df['side']=='buy']['size'].astype('float').sum()

0.195

In [670]:
 db.delete_many( { "MONGOKEY" : "BUY_ORDER" } )

In [514]:
t = pd.to_datetime(time.time()-100,unit='s')
t

Timestamp('2018-08-17 20:35:59.502279999')

In [508]:
df['created_at'][3]

'2018-08-17T20:26:56.739788Z'

In [516]:
openorders = client.get_orders()[0]

In [622]:
df_openorders = pd.DataFrame(openorders)
df_openorders

""


In [621]:
df_openorders = pd.DataFrame(openorders).query('product_id == "BTC-EUR"')
df_openorders

UndefinedVariableError: name 'product_id' is not defined

In [520]:
df_openorders['created_at'] = pd.to_datetime(df_openorders['created_at'])

,created_at,executed_value,fill_fees,filled_size,id,post_only,price,product_id,settled,side,size,status,time_in_force,type
0,2018-08-17 20:37:29.299244,0.0000000000000000,0.0000000000000000,0.00000000,6451f256-a893-44ab-8e87-2004781a902e,False,4000.00000000,BTC-EUR,False,buy,0.00500000,open,GTC,limit


In [668]:
pd.DataFrame(list(db.find({'MONGOKEY':'FBP_UPDATE'})))

,MONGOKEY,_id,ds_fcst_0002,ds_fcst_002,product_id,timestamp,trend_fcst_0002,trend_fcst_002,yhat_fcst_0002,yhat_fcst_002,yhat_lower_fcst_0002,yhat_lower_fcst_002,yhat_upper_fcst_0002,yhat_upper_fcst_002
0,FBP_UPDATE,5b76240413ab1f0001d8940c,2018-08-17 01:26:00,2018-08-17 01:26:00,NaN,1.534469e+09,6306.997060,6356.773610,6306.997060,6356.773610,6132.051410,6338.034912,6469.843063,6375.703768
1,FBP_UPDATE,5b76242713ab1f0001d8940d,2018-08-17 01:26:00,2018-08-17 01:26:00,NaN,1.534469e+09,6306.997060,6356.773610,6306.997060,6356.773610,6142.399889,6337.900144,6472.756167,6375.028388
2,FBP_UPDATE,5b76244a13ab1f0001d8940e,2018-08-17 01:27:00,2018-08-17 01:27:00,NaN,1.534469e+09,6286.782316,6357.216501,6286.782316,6357.216501,6005.158032,6338.441925,6587.105787,6375.589092
3,FBP_UPDATE,5b762468400f86000155152e,2018-08-17 01:27:00,2018-08-17 01:27:00,NaN,1.534469e+09,6212.109638,6356.270813,6212.109638,6356.270813,5888.290439,6336.878098,6559.078596,6376.974452
4,FBP_UPDATE,5b76246b1aba1e00012e253e,2018-08-17 01:27:00,2018-08-17 01:27:00,NaN,1.534469e+09,6212.109638,6356.270813,6212.109638,6356.270813,5883.710736,6336.112153,6553.147420,6376.891130
5,FBP_UPDATE,5b76246dc86bc30001703d45,2018-08-17 01:27:00,2018-08-17 01:27:00,NaN,1.534469e+09,6212.109638,6356.270813,6212.109638,6356.270813,5881.034098,6334.219306,6545.501341,6375.668371
6,FBP_UPDATE,5b7624736fadab00015e3ea8,2018-08-17 01:27:00,2018-08-17 01:27:00,NaN,1.534469e+09,6212.109638,6356.270813,6212.109638,6356.270813,5874.055963,6335.936249,6555.038955,6375.535117
7,FBP_UPDATE,5b76247413ab1f0001d8940f,2018-08-17 01:27:00,2018-08-17 01:27:00,NaN,1.534469e+09,6320.622338,6357.430428,6320.622338,6357.430428,6091.363339,6339.517370,6550.693098,6375.177106
8,FBP_UPDATE,5b76249d13ab1f0001d89410,2018-08-17 01:28:00,2018-08-17 01:28:00,NaN,1.534469e+09,6269.574433,6356.844693,6269.574433,6356.844693,6016.534716,6334.946823,6530.594632,6380.151837
9,FBP_UPDATE,5b7624c513ab1f0001d89411,2018-08-17 01:29:00,2018-08-17 01:29:00,NaN,1.534469e+09,6259.072801,6359.249776,6259.072801,6359.249776,5946.091792,6342.227409,6543.052678,6376.579087


In [667]:

l = []
with db.watch() as stream:
        for change in stream:
            print(change)
            l.append(change)

{'_id': {'_data': '825B775CCD0000000129295A10041D485BD8E5F24A7F97FF3EFC75D0DBDB463C5F6964003C33343834373732642D363332332D346661372D386637612D326532663364363031626334000004'}, 'operationType': 'insert', 'clusterTime': Timestamp(1534549197, 1), 'fullDocument': {'_id': '3484772d-6323-4fa7-8f7a-2e2f3d601bc4', 'type': 'received', 'order_type': 'market', 'size': 0.012, 'side': 'sell', 'funds': 78.63576, 'product_id': 'BTC-USD', 'sequence': 6642208702, 'time': '2018-08-17T23:39:57.677000Z', 'y': 6552.9800000000005, 'timestamp': 1534549197.6827226, 'MONGOKEY': 'MARKET_UPDATE'}, 'ns': {'db': 'gdax', 'coll': 'gdaxws'}, 'documentKey': {'_id': '3484772d-6323-4fa7-8f7a-2e2f3d601bc4'}}
{'_id': {'_data': '825B775CCE0000000129295A10041D485BD8E5F24A7F97FF3EFC75D0DBDB463C5F6964003C32623866353862392D353335362D343836652D623864612D666666643131313038633266000004'}, 'operationType': 'insert', 'clusterTime': Timestamp(1534549198, 1), 'fullDocument': {'_id': '2b8f58b9-5356-486e-b8da-fffd11108c2f', 'type': 'rec

KeyboardInterrupt: 